# Polyfem Python Tutorial

This is a jupyter notebook. The "real" notebook can be found [here](https://github.com/polyfem/polyfem.github.io/blob/docs/docs/python_examples.ipynb).

Polyfem relies on 3 main objects:

1. `Settings` that contains the main settings such discretization order (e.g., $P_1$ or $P_2$), material parameters, formulation, etc.
2. `Problem` that describe the problem you want to solve, that is the boundary conditions and right-hand side. There are some predefined problems, such as `DrivenCavity`, or generic problems, such as `GenericTensor`.
3. `Solver` that is the actual FEM solver.

The usage of specific problems is indented for benchmarking, in general you want to use the `GenericTensor` for tensor-based PDEs (e.g., elasticity) or `GenericScalar` for scalar PDEs (e.g., Poisson). 

A typical use of Polyfem is:

```python
settings = polyfempy.Settings(
    pde=polyfempy.PDEs.LinearElasticity, # or any other PDE
    discr_order=2
)
# set necessary settings
# e.g. settings.discr_order = 2

problem = polyfempy.Problem() # or any other problem
# set problem related data
# e.g. problem.set_displacement(1, [0, 0], [True, False])

settings.problem = problem

#now we can create a solver and solve
solver = polyfempy.Solver()

solver.settings(settings)
solver.load_mesh_from_path(mesh_path, normalize_mesh=False)

solver.solve()
```

**Note 1**: for legacy reasons Polyfem always normalizes the mesh (i.e., rescale it to lay in the $[0,1]^d$ box, you can use `normalize_mesh=False` when loading to disable this feature.

**Note 2**: the solution $u(x)$ of a FEM solver are the coefficients $u_i$ you need to multiply the bases $\varphi_i(x)$ with:
$$
u(x)=\sum u_i \varphi_i(x).
$$
The coefficients $u_i$ are **unrelated** with the mesh vertices because of reordering of the nodes or high-order bases. For instance $P_2$ bases have additional nodes on the edges which do not exist in the mesh.

For this reason Polyfem uses a *visualization mesh* where the solution is sampled at the vertices.
This mesh has two advantages: 
1. it solves the problem of nodes reordering and additional nodes in the same way
2. it provides a "true" visualization for high order solution by densely sampling each element (a $P_2$ solution is a piecewise quadratic function which is visualized in a picewise linear fashion, thus the need of a dense element sampling).

To control the resolution of the visualization mesh use `vismesh_rel_area` while loading.

## Examples

Some imports for plotting

In [1]:
import meshplot as mp
import numpy as np
import polyfempy as pf

### Utility

Creates a quad mesh of `n_pts` x `n_pts` in the form of a regular grid

In [2]:
def create_quad_mesh(n_pts):
    extend = np.linspace(0,1,n_pts)
    x, y = np.meshgrid(extend, extend, sparse=False, indexing='xy')
    pts = np.column_stack((x.ravel(), y.ravel()))
    
    faces = np.ndarray([(n_pts-1)**2, 4],dtype=np.int32)

    index = 0
    for i in range(n_pts-1):
        for j in range(n_pts-1):
            faces[index, :] = np.array([
                j + i * n_pts,
                j+1 + i * n_pts,
                j+1 + (i+1) * n_pts,
                j + (i+1) * n_pts
            ])
            index = index + 1
            
    return pts, faces

## Plate hole

This is the python version of the plate with hole example explained [here](https://polyfem.github.io/tutorial/#boundary-conditions).

Set the mesh path

In [3]:
mesh_path = "plane_hole.obj"

create settings:

- Pick linear $P_1$ elements (if the mesh would be a quad it would be $Q_1$)
- We are use a linear material model

In [4]:
settings = pf.Settings(
    discr_order=1,
    pde=pf.PDEs.LinearElasticity
)

and choose Young's modulus and poisson ratio

In [5]:
settings.set_material_params("E", 210000)
settings.set_material_params("nu", 0.3)

Next we setup the problem

In [6]:
problem = pf.Problem()

sideset 1 has symetric boundary in $x$

In [7]:
problem.set_x_symmetric(1)

sideset 4 has symmetric boundary in $y$

In [8]:
problem.set_y_symmetric(4)

sideset 3 has a force of [100, 0] applied

In [9]:
problem.set_force(3, [100, 0])

fianally set the problem

In [10]:
settings.problem = problem

Solve!
Note: we normalize the mesh to be in $[0,1]^2$

In [11]:
solver = pf.Solver()

solver.settings(settings)
solver.load_mesh_from_path(mesh_path, normalize_mesh=True)

solver.solve()

[2024-07-21 21:11:10.357] [polyfem] [info] Loading mesh...
[2024-07-21 21:11:10.357] [geogram] [info] Loading file plane_hole.obj...
[2024-07-21 21:11:10.372] [geogram] [info] (FP64) nb_v:8549 nb_e:0 nb_f:16797 nb_b:299 tri:1 dim:3
[2024-07-21 21:11:10.372] [geogram] [info] Attributes on vertices: point[3]
[2024-07-21 21:11:10.383] [polyfem] [info] mesh bb min [0, 0], max [1, 0.500001]
[2024-07-21 21:11:10.384] [polyfem] [info]  took 0.026449s
[2024-07-21 21:11:10.385] [polyfem] [info] simplex_count: 	16797
[2024-07-21 21:11:10.385] [polyfem] [info] regular_count: 	0
[2024-07-21 21:11:10.385] [polyfem] [info] regular_boundary_count: 	0
[2024-07-21 21:11:10.385] [polyfem] [info] simple_singular_count: 	0
[2024-07-21 21:11:10.385] [polyfem] [info] multi_singular_count: 	0
[2024-07-21 21:11:10.385] [polyfem] [info] boundary_count: 	0
[2024-07-21 21:11:10.385] [polyfem] [info] multi_singular_boundary_count: 	0
[2024-07-21 21:11:10.385] [polyfem] [info] non_regular_count: 	0
[2024-07-21 21:

Get the solution

In [12]:
pts, tets, disp = solver.get_sampled_solution()

diplace the mesh

In [13]:
vertices = pts + disp

and get the stresses

In [14]:
mises, _ = solver.get_sampled_mises_avg()

finally plot with the above code

In [15]:
mp.plot(vertices, tets, mises, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5002993…

Note that we used `get_sampled_mises_avg` to get the Von Mises stresses because they depend on the Jacobian of the displacement which, becase we use $P_1$ elements, is piece-wise constant. To avoid that effect in `get_sampled_mises_avg` the mises are averaged per vertex weighted by the area of the triangles. If you want the "real" mises just call

In [16]:
mises = solver.get_sampled_mises()
mp.plot(vertices, tets, mises, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5002993…

## Plate hole with High-Order Mesh

This is the same example as before, but we use `wildmeshing` to create a curved mesh.

In [17]:
import wildmeshing as wm
mesh_path = "plane_hole.svg"

v, f, nodes, F_nodes = wm.triangulate_svg(mesh_path, cut_outside=True)

1/1 n sub paths: 5
Loading and preprocessing ...
remove duplicates:
#v 69->64
#e 64->64
simplify
0/5 features are simplified
#features = 5

get_inflections & cut_reflex
#features = 5

sample_features

remove_high_curvature
0 segments are removed
#features = 5

remove_short_features
0 short edges are removed
#features = 5

mu_separation
#pairs = 5
#check_feature = 5(5)
0.014208s
#features = 5

cut_inflections
#features = 5

remove_short_features again
0 short edges are removed
#features = 5

gen_segments
#v = 36
#e = 36

refine:
#v 64->36
#e 64->36
simplify:
#v 36->36
#e 36->36
#secondary_features = 0
Loaded and preprocessed.
time = 0.052303s

BSP subdivision...
#v 120->168
#f 204->300
BSP subdivision done.
time = 0.001875s

Mesh optimization...
#v = 168(168)    unrounded = 0
#t = 300(300)
max_energy = 16.3103
avg_energy = 3.86064
min_angle = 4.39204, max_angle = 132.55
< 5: 0.00333333, < 10: 0.06, < 15: 0.0866667
> 175: 0, > 170: 0, > 165: 0

//////////////// preprocessing ////////////

Now we proceed as before

In [18]:
#create settings
settings = pf.Settings(
    discr_order=1, #pick linear P_1 elements, even if the geometry is P_3
    pde=pf.PDEs.LinearElasticity #Linear elasticity
)

#Material parameters
settings.set_material_params("E", 210000)
settings.set_material_params("nu", 0.3)

In [19]:
problem = pf.Problem()

#sideset 1 is symmetric in x
problem.set_x_symmetric(1)

#sideset 4 is symmetric in y
problem.set_y_symmetric(4)

#sideset 3 has a force of [100, 0] applied
problem.set_force(3, [100, 0])

fianally set the problem

In [20]:
settings.problem = problem

Create the solver and load the high-order mesh, the only difference with respect to before

In [21]:
solver = pf.Solver()

solver.settings(settings)
solver.set_high_order_mesh(v, f, nodes, F_nodes, normalize_mesh=True)

[2024-07-21 21:11:14.961] [polyfem] [info] Loading mesh...
[2024-07-21 21:11:14.961] [polyfem] [info] mesh bb min [0, 0], max [1, 0.5]
[2024-07-21 21:11:14.961] [polyfem] [info]  took 5.7e-05s


And finally, solve!

In [22]:
#solver.solve()

Get and plot the solution, same as before

In [23]:
solver_2_breaks = True

if not solver_2_breaks:
    pts, tets, disp = solver.get_sampled_solution()

    #diplace the mesh
    vertices = pts + disp

    #get the stresses
    mises, _ = solver.get_sampled_mises_avg()

    #plot
    mp.plot(vertices, tets, mises, return_plot=True)

## Torsion

Non-linear example. We want to torque a 3D bar around the $z$ direction.

The example is really similar as the one just above.

Sets the mesh, create a solver, and load the mesh.

In this case the mesh has already the correct size. We also choose a coarse visualization mesh

In [24]:
mesh_path = "square_beam_h.HYBRID"
solver = pf.Solver()
solver.load_mesh_from_path(mesh_path, normalize_mesh=False, vismesh_rel_area=0.001)

[2024-07-21 21:11:15.020] [polyfem] [info] Loading mesh...
[2024-07-21 21:11:15.031] [polyfem] [info] mesh bb min [-10, -10, 0], max [10, 10, 100]
[2024-07-21 21:11:15.031] [polyfem] [info]  took 0.01104s


We want to use the default sideset marking, top of the mesh is 5 and bottom is 6.

Let's verify. We first extract the sidesets: `p` are some point, `t` triangles, and `s` the sidesets from 1 to 6

In [25]:
p, t, s = solver.get_boundary_sidesets()

Now we can plot it

In [26]:
tmp = np.zeros_like(s)
tmp[s==5] = 1
tmp[s==6] = 1

mp.plot(p, t, tmp, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Now we create the settings, as before.

Note: It is an hex-mesh so we are using $Q_1$.

Differently from before we want a non-linear material model: NeoHookean.

To avoid ambiguities in the rotation, we want to do 5 steps of incremental loading.

In [27]:
settings = pf.Settings(
    discr_order=1,
    pde=pf.PDEs.NonLinearElasticity,
    
    nl_solver_rhs_steps=5
)

Choose Young's modulus and Poisson's ratio, as before

In [28]:
settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)

Now we setup problem with fixed sideset (5), rotating sideset (6), ahlf a tour along the $z$-axis.

In [29]:
problem = pf.Torsion(
    fixed_boundary=5, #sideset 5 is fixed
    turning_boundary=6, #sideset 6 rotates
    
    n_turns = 0.5, #by half a tour
    
    axis_coordiante=2, #around the z-axis, 2
    
)

: 

and set the problem and solve

In [30]:
settings.problem = problem

#solving!
solver.settings(settings)

solver.solve()

[2024-07-21 21:11:15.336] [polyfem] [info] simplex_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] regular_count: 	108
[2024-07-21 21:11:15.336] [polyfem] [info] regular_boundary_count: 	356
[2024-07-21 21:11:15.336] [polyfem] [info] simple_singular_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] multi_singular_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] boundary_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] multi_singular_boundary_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] non_regular_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] non_regular_boundary_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] undefined_count: 	0
[2024-07-21 21:11:15.336] [polyfem] [info] total count:	 464
[2024-07-21 21:11:15.336] [polyfem] [info] Building isoparametric basis...
[2024-07-21 21:11:15.343] [polyfem] [info] Computing polygonal basis...
[2024-07-21 21:11:15.343] [polyfem] [info]  took 2.2e-05s
[2024-07-21 21:11:15.344] [polyfem] [info] hmin: 3.4482
[202

takes approx 1 min because it is a complicated non-linear problem!

Get solution and stesses as before

Since we want to show only the surface there is no need of getting the whole volume, so we set `boundary_only` to `True`

In [ ]:
pts, tets, disp = solver.get_sampled_solution(boundary_only=True)
vertices = pts + disp
mises, _ = solver.get_sampled_mises_avg(boundary_only=True)

and plot the 3d result!

In [ ]:
mp.plot(vertices, tets, mises, shading={"flat":True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

## Fluid simulation

Create the mesh using the utility function

In [ ]:
pts, faces = create_quad_mesh(50)

create settings, pick linear $Q_2$ elements for velocity and $Q_1$ for pressure and select stokes as material model.

In [ ]:
settings = pf.Settings(
    discr_order=2,
    pressure_discr_order=1,
    
    pde=pf.PDEs.Stokes
)

Set the viscosity of the fluid

In [ ]:
settings.set_material_params("viscosity", 1)

The default solver do not support mixed formulation, we need to choose `UmfPackLU`

In [ ]:
settings.set_advanced_option("solver_type", "Eigen::UmfPackLU")

We use the standard [Driven Cavity](https://www.cfd-online.com/Wiki/Lid-driven_cavity_problem) problem

In [ ]:
problem = pf.DrivenCavity()

we set the problem

In [ ]:
settings.problem = problem

We create the solver and set the settings

In [ ]:
solver = pf.Solver()
solver.settings(settings)

This time we are using pts and faces instead of loading from the disk

In [ ]:
solver.set_mesh(pts, faces, vismesh_rel_area=0.001)

Solve!

In [ ]:
solver.solve()

We now get the solution and the pressure

In [ ]:
pts, tris, velocity = solver.get_sampled_solution()

and plot it!

In [ ]:
each = 10
p = mp.plot(pts, tris, np.linalg.norm(velocity, axis=1), return_plot=True)
p.add_lines(pts[0:pts.shape[0]:each,:], pts[0:pts.shape[0]:each,:]+velocity[0:pts.shape[0]:each,:]/6)
p

## Time dependent simulation

Create the mesh using the utility function

In [ ]:
pts, faces = create_quad_mesh(50)

create settings, pick linear $Q_1$ elements, and a linear material model.

In this case we want to run a simulation with $t\in[0, 10]$ and have 50 time steps.

In [ ]:
settings = pf.Settings(
    discr_order=1,
    pde=pf.PDEs.LinearElasticity,
    
    tend=10,
    time_steps=50
)

Choose Young's modulus and poisson ratio

In [ ]:
settings.set_material_params("E", 1)
settings.set_material_params("nu", 0.3)

Next we setup the problem, this doesnt have any parameters. It will...

In [ ]:
problem = pf.Gravity()

we set the problem

In [ ]:
settings.problem = problem

We create the solver and set the settings

In [ ]:
solver = pf.Solver()
solver.settings(settings)

This time we are using `pts` and `faces` instead of loading from the disk (For efficienty in the browser we select a coarse vis mesh)

In [ ]:
solver.set_mesh(pts, faces, vismesh_rel_area=0.001)

Solve!

In [ ]:
solver.solve()

Get the solution and the mises

In [ ]:
pts = solver.get_sampled_points_frames()
tris = solver.get_sampled_connectivity_frames()
disp = solver.get_sampled_solution_frames()
mises = solver.get_sampled_mises_avg_frames()

In [ ]:
def plot(frame, p=None):
    return mp.plot(pts[frame]+disp[frame], tris[frame], mises[frame], return_plot=True, plot=p)

Before the animation, let's plot the solution some frames

In [ ]:
plot(0)

In [ ]:
plot(5)

In [ ]:
plot(11)

Now we are ready to do the animation

In [ ]:
p = plot(0)

@mp.interact(frame=(0, len(mises)))
def step(frame=0):
    plot(frame, p)